# TRAINING JUPYTER

## I) DATA

### Data Loading

In [1]:
from generator import generate_conv
from preprocess import DictPrepross, Zero_One, Normalizer
from netCDF4 import Dataset
import numpy as np
import os

datafolder ='Data'
fin = Dataset(os.path.join(datafolder, 'f522_dh.trainingdata_in.lcv.20190401_0000z.nc4'))
fout = Dataset(os.path.join(datafolder, 'f522_dh.trainingdata_out.lcv.20190401_0000z.nc4'))
x = fin.variables
y = fout.variables
n,p = x['Xdim'].shape[0],x['Ydim'].shape[0]

### Parameter setting

In [2]:
batch_size = 64  # size of batch
header = list(x.keys())[6:]  # variables are relevant after "time"
kernels = [] # list of kernels
testprop = 0.9 # proportion of test data
seed = 0;  np.random.seed(seed); # setting the seed

 Creation of the preprocessor to normalize the data

In [3]:

header_dict = ["ql","qi","q","rl","ri","ts","t", "pl","emis","o3"]  # variable we preprocess
fct = [] # how to preprocess them
for i in range(5):
    fct.append(Zero_One())
for j in range(5):
    fct.append(Normalizer())
    
D = DictPrepross(header_dict, fct)
del(fct)
D.fitonNetCDF(x)

fix parameters

In [4]:
d = len(header) + np.sum(np.array( [len(k.header) for k in kernels]  ))
batch_per_epoch = int(n*p / batch_size)

## II) ARCHITECTURES

### Bidir RNN followed by fcn

In [5]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, RNN, SimpleRNN, GRU, Activation, Flatten
from keras import optimizers
from keras.layers import Bidirectional
from keras import backend as K


modelbd = Sequential()
modelbd.add(Bidirectional(LSTM(128, return_sequences=True),input_shape=(72, 11)))
modelbd.add(Dense(50))
modelbd.add(Activation('relu'))
modelbd.add(Dense(20))
modelbd.add(Activation('relu'))
modelbd.add(Dense(1))
modelbd.add(Flatten())
modelbd.add(Dense(73))

rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1.e-5)
modelbd.compile(loss='mse', optimizer=rmsprop)
modelbd.summary()

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 72, 256)           143360    
_________________________________________________________________
dense_1 (Dense)              (None, 72, 50)            12850     
_________________________________________________________________
activation_1 (Activation)    (None, 72, 50)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 72, 20)            1020      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 20)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 72, 1)             21        
_________________________________________________________________
flatten_1 (Flatten)          (None, 72)                0         
__________

In [6]:
train_generator =  lambda Params, batch_size, nb=-1 : generate_conv( \
                            Params[0], Params[1], kernels=kernels,\
                            train_prop=testprop, header=header,\
                            batch_size = batch_size, maxbatch=nb,  preprocess=D, test_data=False)

test_generator = lambda Params, batch_size, nb=1 : generate_conv( \
                            Params[0], Params[1], kernels=kernels,\
                            train_prop=testprop, header=header,\
                            batch_size = batch_size, maxbatch=nb, preprocess=D, test_data=True)

In [7]:
testsize = 128*4
for x0,y0 in test_generator( (x,y), batch_size=testsize, nb=1):
    data_test = (x0,y0)

In [10]:
if len(K.tensorflow_backend._get_available_gpus())>0:
    history = modelbd.fit_generator( train_generator( (x, y), batch_size=64),steps_per_epoch=50, epochs=10, validation_data=data_test)

1

In [82]:
import datetime
import os

model_folder = "TrainedModels"
prefix = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
archi = "Bidir"
if not os.path.isdir(model_folder):
    os.mkdir(model_folder)
modelbd.save(  os.path.join(model_folder, prefix+archi+'.h5')  )

### VISUALISATION

In [97]:
from keras.models import load_model

In [100]:
models = []
for weight in os.listdir("TrainedModels/"):
    models.append(load_model(os.path.join("TrainedModels",weight)))
    ytruth = data_test[1]
    ypred  = models[-1].predict(data_test[0])
    print(weight, np.sum(np.abs(ypred-ytruth))/testsize/72)

20190501152954Bidir.h5 40.024800618489586
20190501155636Bidir.h5 35.029927571614586


In [8]:
import matplotlib.pyplot as plt

def multipred(models,x):
    bs = x.shape[0]
    yp = np.zeros((len(models),bs,73))
    for i,model in enumerate(models):
        yp[i,:,:]=model.predict(x)
    return(yp)

ytruth = data_test[1]
ypred  = multipred(models, data_test[0])

In [ ]:
i=np.random.randint(testsize)
plt.plot(ypred[0,i],np.arange(73)); plt.legend("pred1");
plt.plot(ypred[1,i],np.arange(73)); plt.legend("pred2");
plt.plot( (ypred[1,i]+ypred[0,i])/2,np.arange(73)); plt.legend("pred1");
plt.plot(ytruth[i],np.arange(73)); plt.legend("truth");

### SECOND ARCHITECTURE :
Not tested yet

In [3]:
def CreateConv1D(n_unit, kernel_size, reg=0, seed=0, inputsize=False):
    if not inputsize:
        return(Conv1D(filters=n_unit, kernel_size=kernel_size, \
                 kernel_regularizer=regularizers.l2(reg), \
                 padding="same", kernel_initializer = initializers.normal(seed=seed)))
    else:
        return(Conv1D(input_shape=inputsize, filters=n_unit, kernel_size=kernel_size, \
                 kernel_regularizer=regularizers.l2(reg), \
                 padding="same", kernel_initializer = initializers.normal(seed=seed)))
        

def CreateModel(Hn, Ks, inputsize, reg=0, seed=0):
    model = Sequential()
    model.add(CreateConv1D(Hn[0], Ks[0], reg, seed, inputsize))
    model.add(Activation("relu"))
    for i in range(1,len(Hn)-1):
        model.add(GenerateConv1D(Hn[i], Ks[i], reg, seed, False))
        model.add(Activation("relu"))
    model.add(Flatten())
    model.add(Dense(73, kernel_regularizer = regularizers.l2(0.01),\
                kernel_initializer = initializers.normal(seed=s)))
    return(model)

HNn = [128,64,16,4] # hidden layer size
Ks = [10, 10, 3, 3]
inputsize = (lev, d)

model = CreateModel(HNn, Ks, inputsize, 0.01)

sgd = optimizers.SGD(lr=0., decay=1e-6, momentum=0.95)

model.compile(optimizer=sgd,
             loss='mse',
             )
model.save('arch1_FCNN.h5')

In [2]:
# PARAMETERS 
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Activation,Input,Flatten, UpSampling1D
from keras import optimizers, regularizers, initializers

# PARAMETERS :
batch_size = 64
header = list(x.keys())[6:] # variables are relevant after "time"
kernels = []
testprop = 0.9
s = 0
np.random.seed(s)

n,p, lev = x['Xdim'].shape[0],x['Ydim'].shape[0],x['lev'].shape[0]

d = len(header) + np.sum(np.array( [len(k.header) for k in kernels]  )).astype(int)
batch_per_epoch = int(n*p / batch_size)
batch_generator = lambda Params, batch_size : generate_conv( \
                            Params[0], Params[1], kernels=kernels,\
                            train_prop=testprop, header=header,\
                            batch_size = batch_size)

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# III) TRACKS 

### MODEL 2 :

- FCNN
- (with AE)

##### MODEL 2 : FCM-Final FC

##### MODEL 2 : U-net :
- use regular U-net so all layers affect each other and more stability

### MODEL 3 : Bidir-LSTM 
> Possible alternatives

- use two LSTM to show both impact of superior and inferior layer
- use attention model over it
- use w embeddings before

> TD

- Read git trez
- Read article of Hedge fun